# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
#1
import pandas as pd
import tensorflow as tf
tictactoe= pd.read_csv('/content/tic-tac-toe.csv')

In [2]:
#2
tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
tictactoe.describe()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626


In [4]:
tictactoe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [13]:
#3
from sklearn import preprocessing
labelencoder=preprocessing.LabelEncoder()
labelencoder.fit(tictactoe['class'])
print(labelencoder.classes_)
print(labelencoder.transform(tictactoe['class']))


[False  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 

In [20]:
#4
import numpy as sp
from sklearn.model_selection import train_test_split

def split_train_test(tictactoe, test_size=0.2):
    train_df, test_df= train_test_split(tictactoe, test_size=test_size, random_state=42)
    print(train_df.shape, test_df.shape)
    train_x = train_df.drop(columns=['class'])
    train_y = train_df['class']
    test_x = test_df.drop(columns=['class'])
    test_y = test_df['class']
    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = split_train_test(tictactoe)

(766, 10) (192, 10)


In [21]:
train_x.head() #has catergorical

,TL,TM,TR,ML,MM,MR,BL,BM,BR
302,o,x,x,o,x,o,b,x,b
467,o,b,o,o,x,b,x,x,x
294,x,b,b,b,x,b,o,o,x
548,b,o,x,o,b,x,b,b,x
465,o,b,o,x,o,b,x,x,x


In [24]:
#5 normalizing
from sklearn.preprocessing import MinMaxScaler #bc there aren't any outliers

def get_one_hot_encoded_data(train_x, test_x):
    train_x_encoded = pd.get_dummies(train_x) # one hot encoding when lazy
    test_x_encoded = pd.get_dummies(test_x)
    return train_x_encoded, test_x_encoded



def scale_data(train_x_encoded, test_x_encoded):
    scaler = MinMaxScaler()
    scaler.fit(train_x_encoded)
    train_x_scaled = scaler.transform(train_x_encoded)
    test_x_scaled = scaler.transform(test_x_encoded) #never fit scaler on test
    return train_x_scaled, test_x_scaled

train_x_encoded, test_x_encoded = get_one_hot_encoded_data(train_x, test_x)
train_x_scaled, test_x_scaled = scale_data(train_x_encoded, test_x_encoded)
train_x_scaled

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 0.]])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [49]:
#1
#no need bc we already did it in step 1
#2
import tensorflow as tf
from tensorflow.keras.models import Sequential


In [88]:
#3
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(27,)))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [89]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_17 (Dense)                     │ (None, 50)                  │           1,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 40)                  │           2,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 30)                  │           1,230 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 20)                  │             620 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 2)                   │              42 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,332 (20.83 KB)

 Trainable params: 5,332 (20.83 KB)

 Non-trainable params: 0 (0.00 B)

In [90]:
#4
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)

In [91]:
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [92]:
train_x_scaled.shape

(766, 27)

In [93]:
train_y.shape

(766,)

In [94]:
test_x_scaled.shape, test_y.shape

((192, 27), (192,))

In [95]:
#5
data_fit = model.fit(train_x_scaled, train_y,
                    batch_size=32,
                    epochs=15,
                    verbose=1,
                    validation_data=(test_x_scaled, test_y))

Epoch 1/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6273 - loss: 0.6666 - val_accuracy: 0.6510 - val_loss: 0.6252
Epoch 2/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6719 - loss: 0.5997 - val_accuracy: 0.7240 - val_loss: 0.5780
Epoch 3/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7193 - loss: 0.5566 - val_accuracy: 0.7448 - val_loss: 0.5331
Epoch 4/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7793 - loss: 0.4771 - val_accuracy: 0.7240 - val_loss: 0.5027
Epoch 5/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8065 - loss: 0.4241 - val_accuracy: 0.7917 - val_loss: 0.4541
Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8297 - loss: 0.3758 - val_accuracy: 0.8177 - val_loss: 0.4077
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8699 - loss: 0.3014 - val_accuracy: 0.8490 - val_loss: 0.3436
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9316 - loss: 0.2146 - val_accuracy: 0.8750 - val_loss

In [96]:
#6 + 7
score = model.evaluate(test_x_scaled, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07848089188337326
Test accuracy: 0.9791666865348816


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [97]:
test_y

,class
836,False
477,True
350,True
891,False
855,False
...,...
462,True
673,False
33,True
31,True


In [101]:
print("predicted", model.predict(test_x_scaled[3:4]))
print("actual", test_y.iloc[3])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
predicted [[0.99683917 0.0031608 ]]
actual False


In [83]:
test_x_scaled[3].reshape(1,27)

array([[1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
        1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.]])

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
#yes it acheived low loss and high accuracy
#Test loss: 0.07848089188337326
#Test accuracy: 0.9791666865348816

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here